## Dataset for Head Exercise
C - stands for correct

W - stands for wrong

In [1]:
import pandas as pd
import mediapipe as mp
import os
import cv2, csv
import seaborn as sns
import numpy as np

import warnings
warnings.filterwarnings('ignore')

mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [2]:
IMPORTANT_LMS=[
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT EAR",
    "RIGHT EAR",
    "RIGHT_ELBOW",
    "LEFT_ELBOW",
    "LEFT PINKY",
    "RIGHT PINKY"    
    ]

In [3]:
HEADERS = ["label"] # Label column

for lm in IMPORTANT_LMS:
    HEADERS += [f"{lm.lower()}_x", f"{lm.lower()}_y", f"{lm.lower()}_z", f"{lm.lower()}_v"]

# Important Functions

In [4]:
video_paths=["right_posture.MOV", 'test_right.MOV','test_wrong.MOV',
    "wrong_posture.MOV"]

In [5]:
for path in video_paths:
    if os.path.exists(path):
        print("yes")
    else:
        print('no')

yes
yes
yes
yes


## Extract the Video

In [9]:
# Determine important landmarks for plank
IMPORTANT_LMS = [
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT EAR",
    "RIGHT EAR",
    "RIGHT_ELBOW",
    "LEFT_ELBOW",
    "LEFT PINKY",
    "RIGHT PINKY"    
    ]


# Generate all columns of the data frame

HEADERS = ["label"] # Label column

for lm in IMPORTANT_LMS:
    HEADERS += [f"{lm.lower()}_x", f"{lm.lower()}_y", f"{lm.lower()}_z", f"{lm.lower()}_v"]

In [21]:
def rescale_frame(frame, percent=50):
    '''
    Rescale a frame to a certain percentage compare to its original frame
    '''
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation = cv2.INTER_AREA)
    

def init_csv(dataset_path: str):
    '''
    Create a blank csv file with just columns
    '''

    # Ignore if file is already exist
    if os.path.exists(dataset_path):
        return

    # Write all the columns to a empty file
    with open(dataset_path, mode="w", newline="") as f:
        csv_writer = csv.writer(f, delimiter=",", quotechar='"', quoting=csv.QUOTE_MINIMAL)
        csv_writer.writerow(HEADERS)




    try:
        # Extract coordinates of only IMPORTANT_LMS landmarks
        for lm in IMPORTANT_LMS:
            keypoint = landmarks[mp_pose.PoseLandmark[lm].value]
            keypoints.extend([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])

        # Insert action as the label (first column)
        keypoints.insert(0, action)  

        # Append new row to .csv file
        with open(dataset_path, mode="a", newline="") as f:
            csv_writer = csv.writer(f, delimiter=",", quotechar='"', quoting=csv.QUOTE_MINIMAL)
            csv_writer.writerow(keypoints)

    except Exception as e:
        print(f" Error saving landmarks: {e}")


def describe_dataset(dataset_path: str):
    '''
    Describe dataset
    '''

    data = pd.read_csv(dataset_path)
    print(f"Headers: {list(data.columns.values)}")
    print(f'Number of rows: {data.shape[0]} \nNumber of columns: {data.shape[1]}\n')
    print(f"Labels: \n{data['label'].value_counts()}\n")
    print(f"Missing values: {data.isnull().values.any()}\n")
    
    duplicate = data[data.duplicated()]
    print(f"Duplicate Rows : {len(duplicate.sum(axis=1))}")

    return data


def remove_duplicate_rows(dataset_path: str):
    '''
    Remove duplicated data from the dataset then save it to another files
    '''
    
    df = pd.read_csv(dataset_path)
    df.drop_duplicates(keep="first", inplace=True)
    df.to_csv(f"cleaned_train_head.csv", sep=',', encoding='utf-8', index=False)
    

def concat_csv_files_with_same_headers(file_paths: list, saved_path: str):
    '''
    Concat different csv files
    '''
    all_df = []
    for path in file_paths:
        df = pd.read_csv(path, index_col=None, header=0)
        all_df.append(df)
    
    results = pd.concat(all_df, axis=0, ignore_index=True)
    results.to_csv(saved_path, sep=',', encoding='utf-8', index=False)

In [17]:
DATASET_PATH = "train_head.csv"
VIDEO_PATHS=["right_posture.MOV", 'test_right.MOV','test_wrong.MOV',
    "wrong_posture.MOV"]
VIDEO_LABELS = ["C", 'C','W',"W"]  # "C" = Correct, "W" = Incorrect

# Select only important landmarks
IMPORTANT_LMS = [
    "LEFT_SHOULDER",
    "RIGHT_SHOULDER",
    "LEFT_EAR",
    "RIGHT_EAR",
    "RIGHT_ELBOW",
    "LEFT_ELBOW",
    "LEFT_PINKY",
    "RIGHT_PINKY"    
    ]


# Generate headers
HEADERS = ["label"]
for lm in IMPORTANT_LMS:
    HEADERS += [f"{lm.lower()}_x", f"{lm.lower()}_y", f"{lm.lower()}_z", f"{lm.lower()}_v"]

# Initialize CSV with headers if not exists
def init_csv(dataset_path):
    if not os.path.exists(dataset_path):
        pd.DataFrame(columns=HEADERS).to_csv(dataset_path, index=False)

# Save only selected landmarks
def export_landmark_to_csv(dataset_path, results, label):
    landmarks = results.pose_landmarks.landmark
    keypoints = []

    try:
        # Extract only important landmarks
        for lm in IMPORTANT_LMS:
            keypoint = landmarks[getattr(mp_pose.PoseLandmark, lm).value]
            keypoints.extend([keypoint.x, keypoint.y, keypoint.z, keypoint.visibility])

        # Insert action as label
        keypoints.insert(0, label)

        # Append row to CSV
        df = pd.DataFrame([keypoints], columns=HEADERS)
        df.to_csv(dataset_path, mode='a', header=False, index=False)

    except Exception as e:
        print(f"Error saving landmarks: {e}")

# Initialize CSV
init_csv(DATASET_PATH)

# Process each video
for video_path, label in zip(VIDEO_PATHS, VIDEO_LABELS):
    if not os.path.exists(video_path):
        print(f"Video not found: {video_path}")
        continue

    cap = cv2.VideoCapture(video_path)
    save_counts = 0

    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():
            ret, image = cap.read()
            if not ret:
                break

            # Resize frame
            image = cv2.resize(image, (640, 480))
            image = cv2.flip(image, 1)

            # Convert BGR to RGB for MediaPipe
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
            results = pose.process(image)

            if not results.pose_landmarks:
                print(f"Cannot detect pose in {video_path}")
                continue

            # Convert back to BGR for OpenCV
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

            # Draw landmarks
            mp_drawing.draw_landmarks(
                image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                mp_drawing.DrawingSpec(color=(244, 117, 66), thickness=2, circle_radius=4),
                mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2))

            # Save selected landmarks with label
            export_landmark_to_csv(DATASET_PATH, results, label)
            save_counts += 1

            # Display video with pose landmarks
            cv2.putText(image, f"Saved: {save_counts}", (50, 50),
                        cv2.FONT_HERSHEY_COMPLEX, 2, (0, 0, 0), 2, cv2.LINE_AA)
            cv2.imshow("Pose Detection", image)

            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    cap.release()
    cv2.destroyAllWindows()

# Fix MacOS window closing issue
for i in range(1, 5):
    cv2.waitKey(1)

print(f"Dataset saved to {DATASET_PATH}")


Cannot detect pose in right_posture.MOV
Cannot detect pose in right_posture.MOV
Cannot detect pose in right_posture.MOV
Cannot detect pose in right_posture.MOV
Cannot detect pose in right_posture.MOV
Cannot detect pose in right_posture.MOV
Cannot detect pose in right_posture.MOV
Cannot detect pose in right_posture.MOV
Cannot detect pose in right_posture.MOV
Cannot detect pose in right_posture.MOV
Cannot detect pose in right_posture.MOV
Cannot detect pose in right_posture.MOV
Cannot detect pose in right_posture.MOV
Cannot detect pose in right_posture.MOV
Cannot detect pose in right_posture.MOV
Cannot detect pose in right_posture.MOV
Cannot detect pose in right_posture.MOV
Cannot detect pose in right_posture.MOV
Cannot detect pose in right_posture.MOV
Cannot detect pose in right_posture.MOV
Cannot detect pose in right_posture.MOV
Cannot detect pose in right_posture.MOV
Cannot detect pose in right_posture.MOV
Cannot detect pose in right_posture.MOV
Cannot detect pose in right_posture.MOV


In [18]:
# csv_files = [os.path.join("./", f) for f in os.listdir("./") if "csv" in f]

# concat_csv_files_with_same_headers(csv_files, "train.csv")

df = pd.read_csv("train_head.csv")
df

,label,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_v,right_shoulder_x,right_shoulder_y,right_shoulder_z,right_shoulder_v,left ear_x,...,left_elbow_z,left_elbow_v,left pinky_x,left pinky_y,left pinky_z,left pinky_v,right pinky_x,right pinky_y,right pinky_z,right pinky_v
0,C,0.507061,0.188322,-0.232415,0.996542,0.873857,0.125780,-0.047860,0.997773,0.575193,...,-0.359485,0.729211,0.517626,1.463686,-0.230664,0.218081,0.630494,1.330310,0.340658,0.091770
1,C,0.500991,0.191071,-0.353564,0.996172,0.919209,0.131183,0.044790,0.997772,0.578591,...,-0.442765,0.689492,0.452132,1.416300,-0.227603,0.208721,0.671629,1.299045,0.442576,0.093819
2,C,0.500645,0.193920,-0.388123,0.995790,0.829298,0.164692,0.224023,0.997749,0.572123,...,-0.512005,0.649138,0.445445,1.384371,-0.292925,0.201347,0.666952,1.269637,0.511914,0.094115
3,C,0.500677,0.193998,-0.509327,0.995467,0.752489,0.177913,0.402088,0.997755,0.545923,...,-0.662576,0.620954,0.409739,1.363640,-0.433341,0.194487,0.649672,1.275249,0.601808,0.091709
4,C,0.517471,0.218314,-0.196028,0.992725,0.727162,0.233420,0.403442,0.997248,0.585782,...,-0.337349,0.571462,0.425940,1.320362,-0.231307,0.180090,0.524587,1.271521,0.686323,0.091815
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11079,W,1.319917,0.886739,-0.552281,0.783838,1.166086,0.996985,-1.375398,0.794292,1.295479,...,0.035895,0.073145,1.265373,0.813705,0.347067,0.069839,1.484230,0.767195,-0.616615,0.090954
11080,W,1.414088,0.883877,-0.602204,0.764051,1.211977,0.953843,-1.369751,0.776717,1.397416,...,-0.077884,0.069915,1.306691,0.790708,0.265772,0.066848,1.528325,0.803288,-0.495818,0.086403
11081,W,1.401357,0.843289,-0.556775,0.745219,1.229807,0.895155,-1.333216,0.757634,1.393942,...,0.001722,0.066510,1.315565,0.717368,0.352428,0.064093,1.525758,0.688808,-0.533264,0.083794
11082,W,1.459433,0.832686,-0.585612,0.726670,1.040238,0.705599,-1.327325,0.747419,1.435673,...,-0.049874,0.063185,1.333323,0.769836,0.283082,0.060503,1.495027,0.734252,-0.512688,0.079886


In [22]:
remove_duplicate_rows("./train_head.csv")

In [25]:
df1=pd.read_csv('cleaned_train_head.csv')
len(df1)

11084

## Preprocessing

In [26]:
from sklearn.preprocessing import StandardScaler

In [27]:

# Categorizing label
df1.loc[df1["label"] == "C", "label"] = 0 # Correct 0, wrong 1
df1.loc[df1["label"] == "W", "label"] = 1

In [28]:
df1['label'].astype

<bound method NDFrame.astype of 0        0
1        0
2        0
3        0
4        0
        ..
11079    1
11080    1
11081    1
11082    1
11083    1
Name: label, Length: 11084, dtype: object>

In [29]:
df1['label']=df1['label'].astype('int')

In [31]:
print(df1['label'].sum()) # 11084 samples are in the dataset, and 4129 are labeled as wrong, which is good enough

4129


In [32]:
sc=StandardScaler()

In [33]:

# Categorizing label
df1.loc[df1["label"] == "C", "label"] = 0 # Correct 0, wrong 1
df1.loc[df1["label"] == "W", "label"] = 1

In [35]:
x = df1.drop("label", axis=1)  # Remove "label"
y = df1["label"].astype('int')  # Convert label to integer type


In [36]:
x=pd.DataFrame(sc.fit_transform(x),columns=x.columns)

In [37]:
x.head()

,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_v,right_shoulder_x,right_shoulder_y,right_shoulder_z,right_shoulder_v,left ear_x,left ear_y,...,left_elbow_z,left_elbow_v,left pinky_x,left pinky_y,left pinky_z,left pinky_v,right pinky_x,right pinky_y,right pinky_z,right pinky_v
0,-1.941100,-4.209653,0.196638,-0.169513,9.807836,-5.665634,0.558027,-0.016066,0.708496,-5.431836,...,-0.010211,-4.815550,-0.937622,2.526479,0.995569,-2.893619,2.553290,2.207710,1.937945,-3.074985
1,-2.056765,-4.163305,-0.369117,-0.202321,10.726254,-5.571473,0.938976,-0.016077,0.774878,-4.676510,...,-0.385160,-5.589678,-1.565530,2.372921,1.004532,-2.940994,2.976702,2.101824,2.153325,-3.065577
2,-2.063358,-4.115269,-0.530509,-0.236084,8.905473,-4.987511,1.675926,-0.018305,0.648526,-3.945045,...,-0.696899,-6.376175,-1.629641,2.269452,0.813261,-2.978320,2.928567,2.002226,2.299853,-3.064216
3,-2.062742,-4.113957,-1.096519,-0.264681,7.350005,-4.757119,2.408072,-0.017718,0.136642,-3.978986,...,-1.374809,-6.925481,-1.971967,2.202273,0.402111,-3.013046,2.750691,2.021233,2.489823,-3.075266
4,-1.742717,-3.703955,0.366561,-0.507499,6.837111,-3.789795,2.413640,-0.065153,0.915371,-1.850387,...,0.089450,-7.890087,-1.816641,2.062027,0.993686,-3.085917,1.463139,2.008608,2.668426,-3.074782


## Model Creation

In [51]:
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC

from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_score, accuracy_score, f1_score, recall_score, confusion_matrix
from sklearn.calibration import CalibratedClassifierCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier


In [39]:
X_train,X_test,y_train,y_test=train_test_split(x,y,shuffle=True,random_state=1234,test_size=0.2)

In [41]:
X_train[:5]

,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_v,right_shoulder_x,right_shoulder_y,right_shoulder_z,right_shoulder_v,left ear_x,left ear_y,...,left_elbow_z,left_elbow_v,left pinky_x,left pinky_y,left pinky_z,left pinky_v,right pinky_x,right pinky_y,right pinky_z,right pinky_v
272,-0.726773,-0.356660,1.922539,0.080662,0.230407,-1.380582,-0.550729,0.112118,-0.206037,-0.591288,...,2.409137,-1.794564,0.004698,0.864901,1.772066,-1.196727,1.600201,-1.365945,-1.239296,1.032864
4254,-0.255127,-0.564791,0.404592,0.115665,-0.041057,-0.999559,0.469763,0.160663,-0.100850,-0.972815,...,0.635024,0.180219,0.335614,1.092503,0.631467,-0.213286,0.007007,-1.347632,-1.166928,0.950548
10736,0.230811,3.430349,-1.286284,0.121829,0.259411,3.631546,-0.822325,0.177742,-0.160479,2.868223,...,-1.862446,0.356410,-0.517989,-0.107559,-1.489507,1.035893,-0.237856,-0.144482,-1.007632,1.006255
2570,-0.415076,-0.520292,-0.081378,0.003468,0.056058,-0.121875,0.454457,0.124088,-0.674985,0.390602,...,-0.565685,0.315513,-0.777624,-1.534345,-0.498648,0.675214,-0.066868,0.684248,1.903914,-0.827101
7052,0.730969,1.002621,-2.016714,-0.682427,-0.203042,1.140837,-2.133972,-1.546864,0.802271,0.985243,...,-0.448913,-0.186175,0.572307,-0.007798,0.820295,-3.221374,0.022947,0.050165,1.112571,-3.344900


In [42]:
y_test.shape

(2217,)

In [43]:
y_train.shape

(8867,)

In [44]:
y_train[:5]

272      0
4254     0
10736    1
2570     0
7052     1
Name: label, dtype: int32

In [47]:
y_train.sum() #3286 samples of incorrect posture out of 8867 samples

3286

In [49]:
y_test.shape

(2217,)

In [52]:
y_test.sum() #843 samples of incorrect posture out of 2217

843

In [53]:
algorithms =[("LR", LogisticRegression()),
         ("SVC", SVC(probability=True)),
         ('KNN',KNeighborsClassifier()),
         ("DTC", DecisionTreeClassifier()),
         ("SGDC", CalibratedClassifierCV(SGDClassifier())),
         ("NB", GaussianNB()),
         ('RF', RandomForestClassifier()),]

models = {}
final_results = []
def round_up_metric_results(results) -> list:
    '''Round up metrics results such as precision score, recall score, ...'''
    return list(map(lambda el: round(el, 3), results))


for name, model in algorithms:
    trained_model = model.fit(X_train, y_train)
    models[name] = trained_model

    # Evaluate model
    model_results = model.predict(X_test)

    p_score = precision_score(y_test, model_results, average=None, labels=[0, 1])
    a_score = accuracy_score(y_test, model_results)
    r_score = recall_score(y_test, model_results, average=None, labels=[0, 1])
    f1_score_result = f1_score(y_test, model_results, average=None, labels=[0, 1])
    cm = confusion_matrix(y_test, model_results, labels=[0, 1])
    final_results.append(( name,  round_up_metric_results(p_score), a_score, round_up_metric_results(r_score), round_up_metric_results(f1_score_result), cm))

# Sort results by F1 score
final_results.sort(key=lambda k: sum(k[4]), reverse=True)
pd.DataFrame(final_results, columns=["Model", "Precision Score", "Accuracy score", "Recall Score", "F1 score", "Confusion Matrix"])

,Model,Precision Score,Accuracy score,Recall Score,F1 score,Confusion Matrix
0,RF,"[1.0, 0.995]",0.998196,"[0.997, 1.0]","[0.999, 0.998]","[[1370, 4], [0, 843]]"
1,KNN,"[0.993, 0.974]",0.985566,"[0.984, 0.988]","[0.988, 0.981]","[[1352, 22], [10, 833]]"
2,DTC,"[0.982, 0.962]",0.974290,"[0.977, 0.97]","[0.979, 0.966]","[[1342, 32], [25, 818]]"
3,SVC,"[0.949, 0.938]",0.944971,"[0.963, 0.916]","[0.956, 0.927]","[[1323, 51], [71, 772]]"
4,SGDC,"[0.802, 0.798]",0.801083,"[0.901, 0.638]","[0.849, 0.709]","[[1238, 136], [305, 538]]"
5,LR,"[0.787, 0.779]",0.784393,"[0.894, 0.605]","[0.837, 0.681]","[[1229, 145], [333, 510]]"
6,NB,"[0.659, 0.595]",0.649075,"[0.9, 0.241]","[0.761, 0.343]","[[1236, 138], [640, 203]]"


## Saving the model

In [54]:
import pickle

In [56]:
filename='head_exercises.ipynb'
pickle.dump(model,open(filename,'wb'))